**Instruction File: Building a Parametric Measure of Fixed-Income Securities Library**  

---

### **1. Library Overview**
This library provides tools to price bonds (both zero-coupon and coupon-bearing) and calculate key risk metrics:
- **Pricing**: Bond price, yield-to-maturity (YTM)  
- **Risk Metrics**: Macaulay/modified duration, Value-at-Risk (VaR), Expected Shortfall (ES)  
- **Portfolio Analysis**: Aggregate metrics for bond portfolios  

---

### **2. Core Components**
#### **2.1 Class 1: `FixedIncomePrmSingle` (Single Bond Analysis)**
**Purpose**: Price individual bonds and calculate risk metrics.  

##### **Key Methods & Formulas**  
**1. `__init__()`**  
- **Inputs**:  
  - `yield_curve_today`: DataFrame with monthly yields (columns = 1-360 months, row = rates as percentages).  
  - `maturity`: Bond maturity in years (integer or 0.5 increments for semi-annual alignment).  
  - `is_zcb`: Boolean for zero-coupon bond.  
  - `coupon_rate`, `semi_annual_payment`: Required for coupon bonds.  
- **Output**: Initialized bond object.  

**2. `fit()`**  
*Calculates bond fundamentals.*  
- **Zero-Coupon Bond (ZCB)**:  
  ```python
  Price = Face Value / (1 + Yield)^Maturity
  Macaulay Duration = Maturity
  Modified Duration = Macaulay Duration / (1 + Yield)
  ```
- **Coupon Bond**:  
  ```python
  Price = Σ [CashFlow_t / (1 + Yield_t)^(t/12)]  
  YTM = IRR where NPV = Price (solved via Newton-Raphson)  
  Macaulay Duration = Σ [PV(CashFlow_t) * t] / Price  
  Modified Duration = Macaulay Duration / (1 + YTM/frequency)  
  ```  
  *Frequency = 1 (annual) or 2 (semi-annual).*

**3. `DurationNormal()`**  
*VaR/ES using duration mapping.*  
- **Formula**:  
  ```python
  ΔPrice ≈ -Price * ModifiedDuration * ΔYield  
  VaR = Quantile(α) * |Position| * σ(ΔYield)  
  ES = E[Loss | Loss > VaR]  
  ```  
- **Inputs**: Historical yield curve, volatility model (`simple`, `garch`, `ewma`), interval (days), α.  
- **Output**: Dictionary with `VaR`, `ES`, mapped position, and volatility.  

**4. `HistoricalSimulation()`**  
*VaR/ES using historical yield changes.*  
- **Steps**:  
  1. Interpolate yield curve to daily granularity.  
  2. Simulate yield shifts over `interval`.  
  3. Compute P&L for each scenario.  
  4. Extract VaR/ES from loss distribution.  

---

#### **2.2 Class 2: `FixedIncomePrmPort` (Portfolio Analysis)**  
**Purpose**: Analyze portfolios of bonds with mixed maturities/types.  

##### **Key Methods**  
**1. `__init__()`**  
- **Inputs**: Arrays for `maturity_s`, `num_assets_s`, `face_value_s`, `is_zcb_s`, etc.  
- **Validation**: All arrays must have equal lengths; semi-annual payments required for fractional maturities.  

**2. `fit()`**  
*Aggregates portfolio metrics.*  
- **Formulas**:  
  ```python
  Portfolio Duration = Σ (Weight_i * Duration_i)  
  Weight_i = |Position_i| / Total Portfolio Value  
  ```  
- **Output**: Summary DataFrame with bond-level and portfolio metrics.  

**3. `DurationNormal()`**  
*Portfolio VaR/ES using weighted duration.*  
- Same logic as single bond, but uses portfolio-level modified duration.  

**4. `HistoricalSimulation()`**  
*Full portfolio scenario analysis.*  
- Computes P&L for all bonds simultaneously under historical yield shifts.  

---

### **3. Implementation Steps**  
#### **Step 1: Setup Project Structure**  
```
fixed_income_lib/  
├── __init__.py  
├── bond_pricing.py      # FixedIncomePrmSingle ✅ 
├── portfolio.py         # FixedIncomePrmPort  ✅
├── utils/  
│   ├── yield_curve.py   # Yield curve generators ✅ 
│   └── validation.py    # Input checks (e.g., check_maturity())  ✅
└── tests/               # Unit tests  
```

#### **Step 2: Implement Core Functions**  
1. **Yield Curve Handling**  ✅
   - Create functions to:  
     - Generate monthly yield curves (e.g., `create_flat_yield_curve(base_rate=5.0)`).  
     - Interpolate using cubic splines for fractional maturities.  

2. **Bond Pricing Engine**  
   - For coupon bonds:  
     ```python  
     def calculate_cash_flows(face_value, coupon_rate, maturity, semi_annual):  ✅
         # Returns array of (timing, cashflow)  
     ```  
   - For YTM calculation:  
     ```python  
     def newton_raphson_ytm(cash_flows, price):  ✅
         # Solves NPV = Price iteratively  
     ```  

3. **Risk Calculation**  
   - Volatility models:  
     ```python  
     def simple_volatility(delta_y, interval):  ✅
         return np.std(delta_y) * sqrt(interval)  
     ```  

#### **Step 3: Input Validation**  
Example in `validation.py`:  
```python  
def check_maturity(maturity: float) -> float:  
    if (maturity * 2) % 1 != 0:  
        raise ValueError("Maturity must be in 0.5-year increments.")  
    return maturity  
```  

---

### **4. Testing & Validation**  
1. **Unit Tests**  
   - Test ZCB pricing against manual calculations.  
   - Verify duration values for known cash flow structures.  
   - Validate VaR consistency across volatility models.  

2. **Example Usage**  
```python  
# Single bond  
zcb = FixedIncomePrmSingle(  
    yield_curve_today=yield_curve_df,  
    maturity=5,  
    face_value=1000,  
    is_zcb=True  
)  
print(zcb.bond_price)  # Expected: ~783.53 for 5% yield  

# Portfolio  
portfolio = FixedIncomePrmPort(  
    yield_curve_today=yield_curve_df,  
    maturity_s=np.array([2, 5]),  
    num_assets_s=np.array([100, -50]),  
    is_zcb_s=np.array([True, False])  
)  
print(portfolio.summary)  
```  

---

### **5. Deliverables**  
1. Fully documented classes with type hints.  
2. Unit tests covering edge cases (e.g., maturity alignment, negative rates).  
3. Example Jupyter notebook demonstrating usage.  

**Formula Cheat Sheet**:  
- **Macaulay Duration**: \( \text{MacD} = \frac{\sum_{t=1}^T t \cdot PV(CF_t)}{\text{Price}} \)  
- **Modified Duration**: \( \text{ModD} = \frac{\text{MacD}}{1 + \frac{YTM}{m}} \)  
- **VaR (Duration-Normal)**: \( \text{VaR} = Z_{\alpha} \cdot \text{Price} \cdot \text{ModD} \cdot \sigma(\Delta y) \)  

--- 